In [ ]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.debugger import Debugger3
from debugger.tree_node import NodeRef
from evaluator import BenchViewer
from evaluator import DebuggerStatus
from benchmark_consts import *
from utils.system_utils import list_smt2_files, get_name_hash

In [2]:
qs = UNSTABLE_MARIPOSA
# qs = [q for q in qs if get_name_hash(q) not in LIKELY_FIXED]
# print(len(qs))
mv = BenchViewer(qs)
# vv = BenchViewer(UNSTABLE_VERUS)

[INFO] [init] dbg root: dbg/f986501402/ 


[INFO] [init] dbg root: dbg/1bcd3aa5cb/ [INFO] [init] dbg root: dbg/6737d567f9/ 

[INFO] [init] dbg root: dbg/1a397d663b/ 
[INFO] [init] dbg root: dbg/179f62e2d4/ 
[INFO] [init] dbg root: dbg/83c57f0bcb/ [INFO] [init] dbg root: dbg/59dd7b549d/ [INFO] [init] dbg root: dbg/b546a78766/ 


[INFO] [init] dbg root: dbg/17b0b45c82/ [INFO] [init] dbg root: dbg/2a3b1202bd/ 

[INFO] [init] dbg root: dbg/563f5abfa4/ [INFO] [init] dbg root: dbg/6a7bc24676/ [INFO] [init] dbg root: dbg/ab307c9008/ 




[ERROR] no such project group singleton_563f5abfa4 


[INFO] [init] dbg root: dbg/1b7f64e95b/ 
[INFO] [init] dbg root: dbg/298f3245ca/ [INFO] [init] dbg root: dbg/90e97b47ee/ 



[ERROR] no such project group singleton_90e97b47ee 


...

[INFO] [init] dbg root: dbg/5fbc06f886/ 
[INFO] [init] dbg root: dbg/a878753336/ 
[INFO] [init] dbg root: dbg/2606c5a287/ 
[INFO] [init] dbg root: dbg/91f1dee2f1/ 
[INFO] [init] dbg root: dbg/4c2945c620/ 
[INFO] [init] dbg root: dbg/d29cd9f10a/ 
[INFO] [init] dbg root: dbg/87abcdd247/ 
[INFO] [init] dbg root: dbg/112938db3c/ 
[INFO] [init] dbg root: dbg/5af6d28737/ 
[INFO] [init] dbg root: dbg/270b8c3ed1/ 
[INFO] [init] dbg root: dbg/443cdbaf14/ 
[INFO] [init] dbg root: dbg/0711a2f033/ 
[INFO] [init] dbg root: dbg/e5c19b8ea5/ 
[INFO] [init] dbg root: dbg/db5b85130e/ 
[INFO] [init] dbg root: dbg/b5f812a861/ 
[INFO] [init] dbg root: dbg/862cb1068a/ 
[INFO] [init] dbg root: dbg/085408afeb/ 
[INFO] [init] dbg root: dbg/f5dc6f777b/ [INFO] [init] dbg root: dbg/de19cf9fa2/ 

[INFO] [init] dbg root: dbg/e65b46b51f/ 
[INFO] [init] dbg root: dbg/7f97f9269e/ 
[INFO] [init] dbg root: dbg/e1c4753770/ 
[INFO] [init] dbg root: dbg/ac4102a2e0/ 
[INFO] [init] dbg root: dbg/5d39862f72/ 
[INFO] [init] db

[ERROR] no such project group singleton_f6db5998ec 


[INFO] [init] dbg root: dbg/90270fc66a/ 
[INFO] [init] dbg root: dbg/94bc3cfd5f/ 
[INFO] [init] dbg root: dbg/7e492ef45f/ 
[INFO] [init] dbg root: dbg/95fa9421b7/ 
[INFO] [init] dbg root: dbg/7d2782e41a/ 
[INFO] [init] dbg root: dbg/0ef92feb14/ 
[INFO] [init] dbg root: dbg/473fa6f229/ 


[INFO] [init] dbg root: dbg/58d93cd4ed/ 
[INFO] [init] dbg root: dbg/44ac0b7ce8/ 
[INFO] [init] dbg root: dbg/9796f7a39f/ 
[INFO] [init] dbg root: dbg/a854e6c8ad/ [INFO] [init] dbg root: dbg/d347f45fa2/ 
[INFO] [init] dbg root: dbg/a19f4272bf/ 
[INFO] [init] dbg root: dbg/f7f46a3a04/ 
[INFO] [init] dbg root: dbg/3d6f2d9f7a/ 
[INFO] [init] dbg root: dbg/786d3d1ed6/ 
[INFO] [init] dbg root: dbg/125461812b/ 
[INFO] [init] dbg root: dbg/1c167aafa9/ 
[INFO] [init] dbg root: dbg/8a59d4481f/ [INFO] [init] dbg root: dbg/50b414ceb9/ 

[INFO] [init] dbg root: dbg/09f0c73516/ 
[INFO] [init] dbg root: dbg/3ac32afb30/ 
[INFO] [init] dbg root: dbg/adf9eda1ef/ 
[INFO] [init] dbg root: dbg/7dccaece2d/ 
[INFO] [init] d

[ERROR] no such project group singleton_ab52dadac1 


[INFO] [init] dbg root: dbg/712b931174/ 
[INFO] [init] dbg root: dbg/11e4bbf583/ 
[INFO] [init] dbg root: dbg/eec60fd21e/ 
[INFO] [init] dbg root: dbg/b920f021f6/ 
[INFO] [init] dbg root: dbg/82f91d0d6f/ 
[INFO] [init] dbg root: dbg/22188d7780/ 
[INFO] [init] dbg root: dbg/b231954c65/ 
[INFO] [init] dbg root: dbg/1fc9b21323/ 
[INFO] [init] dbg root: dbg/2cbf19182c/ 
[INFO] [init] dbg root: dbg/81df0e63de/ 
[INFO] [init] dbg root: dbg/5add81d45d/ 
[INFO] [init] dbg root: dbg/f9f9100cfb/ 
[INFO] [init] dbg root: dbg/ed978bf127/ 
[INFO] [init] dbg root: dbg/cb725a5d7b/ 
[INFO] [init] dbg root: dbg/10b27f0984/ 
[INFO] [init] dbg root: dbg/4aecdc3251/ 
[INFO] [init] dbg root: dbg/f8271123d9/ [INFO] [init] dbg root: dbg/b100de24e1/ 

[INFO] [init] dbg root: dbg/8cd183cebe/ 
[INFO] [init] dbg root: dbg/12c7d523b1/ [INFO] [init] dbg root: dbg/fa548b2af5/ [INFO] [init] dbg root: dbg/842038cbbb/ 
[INFO] [init] dbg root: dbg/9b55212ceb/ 

[INFO] [init] dbg root: dbg/167389a8ef/ 
[INFO] [init] dbg

In [4]:
mv.status.print_status()

| category                                      |   count | percentage   |
|-----------------------------------------------|---------|--------------|
| DebuggerStatus.FINISHED                       |     510 | 93.58 %      |
| DebuggerStatus.NO_PROOF                       |      18 | 3.3 %        |
| DebuggerStatus.SINGLETON_CREATION_FAILED      |       9 | 1.65 %       |
| DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED |       8 | 1.47 %       |
| total                                         |     545 | 100.00 %     |


In [ ]:
for q in mv.status[DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED]:
    print("./src/debugger3.py --create-singleton -i", q)

for q in mv.status[DebuggerStatus.SINGLETON_CREATION_FAILED]:
    print("./src/debugger3.py --create-singleton -i", q)

In [ ]:
for q in mv.status[DebuggerStatus.SINGLETON_NOT_RAN]:
# for q in mv.status[DebuggerStatus.SINGLETON_TESTED_EMPTY]:
    assert len(list_smt2_files(mv[q].singleton_dir)) != 0
    print("./src/exper_wizard.py manager -e verify --total-parts 12 -i ", mv[q].singleton_dir, "--clear")
    print("./src/analysis_wizard.py singleton -e verify -s z3_4_13_0 -i ", mv[q].singleton_dir)
    print("python3 src/carve_and_rerun.py", mv[q].filtered_dir)

In [9]:
import numpy as np
# print(len(mv.fixable)/len(mv.status[DebuggerStatus.FINISHED]))

bad_min_ranks = dict()
min_ranks = []

for q in mv.fixable:
    r = mv.reviewers[q]
    report = r.build_report()
    indices = report.freq.loc[report.freq["qname"].isin(report.stabilized["qname"])].index
    report.freq["rank"] = report.freq["trace_count"].rank(method='min', ascending=False)
    min_rank = report.freq.loc[indices]["rank"].min()
    max_rank = report.freq.loc[indices]["rank"].max()
    min_ranks.append(min_rank)
    if min_rank <= 10:
        continue
    bad_min_ranks[q] = (min_rank, len(indices))

min_ranks = np.array(min_ranks)

for q, (rank, feasible)  in bad_min_ranks.items():
    print(int(rank), feasible, q)

11 2 data/projs/bench_unstable/base.z3/d_lvbkv--BlockCacheSystem-JournalInterval.i.dfy.Impl__JournalIntervals.__default.Disk__Journal__append.smt2
36 1 data/projs/bench_unstable/base.z3/d_lvbkv--BlockCacheSystem-BlockSystem.i.dfy.Impl__BlockSystem.__default.DirtyStepPreservesInv.smt2
54 3 data/projs/bench_unstable/base.z3/d_fvbkv--lib-Buckets-BucketsLib.i.dfy.Impl__BucketsLib.__default.BInternalWFBucket.smt2
108 4 data/projs/bench_unstable/base.z3/d_lvbkv--PivotBetree-PivotBetree.i.dfy.Impl__PivotBetree.__default.GCStepRefines.smt2
43 2 data/projs/bench_unstable/base.z3/d_fvbkv--Impl-IndirectionTableImpl.i.dfy.Impl__IndirectionTableImpl.IndirectionTable.FindRefWithNoLoc.smt2
468 1 data/projs/bench_unstable/base.z3/d_fvbkv--Impl-SuccModel.i.dfy.Impl__SuccModel.__default.lemmaGetPathResult.smt2
18 2 data/projs/bench_unstable/base.z3/d_lvbkv--lib-Checksums-F2_X_Lemmas.i.dfy.Impl__F2__X__Lemmas.__default.mod__reverse__mul__reverse.smt2
289 2 data/projs/bench_unstable/base.z3/d_lvbkv--Impl-

In [10]:
def fmt_ratio(x, y):
    return f"({round(x / y * 100, 1)}%)"

def fmt_ratio_2(x, y):
    return f"{x}/{y} ({round(x / y * 100, 1)}%)"

def count_leq(r, n):
    return np.where(r <= n)[0].shape[0]

fixable_count = len(mv.fixable)
finished_count = len(mv.status[DebuggerStatus.FINISHED])
bench_total = mv.status.total
print("finished", fmt_ratio_2(finished_count, bench_total))
print("\tfixable", fmt_ratio_2(fixable_count, finished_count))
rcount = count_leq(min_ranks, 1)
print("\t\t", "top-1 ", fmt_ratio_2(rcount, fixable_count))
rcount = count_leq(min_ranks, 3)
print("\t\t", "top-3 ", fmt_ratio_2(rcount, fixable_count))
rcount = count_leq(min_ranks, 10)
print("\t\t", "top-10", fmt_ratio_2(rcount, fixable_count))
print("\tno-fixes", fmt_ratio_2(finished_count - fixable_count, finished_count))
no_proof = len(mv.status[DebuggerStatus.NO_PROOF])
print("no-proof", fmt_ratio_2(no_proof, bench_total))
parse_fail = len(mv.status[DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED]) + len(mv.status[DebuggerStatus.SINGLETON_CREATION_FAILED])
print("parse-fail", fmt_ratio_2(parse_fail, bench_total))
in_progress = bench_total - finished_count - parse_fail - no_proof
print("in-progress", fmt_ratio_2(in_progress, bench_total))

finished 510/545 (93.6%)
	fixable 375/510 (73.5%)
		 top-1  142/375 (37.9%)
		 top-3  175/375 (46.7%)
		 top-10 241/375 (64.3%)
	no-fixes 135/510 (26.5%)
no-proof 18/545 (3.3%)
parse-fail 17/545 (3.1%)
in-progress 0/545 (0.0%)


In [ ]:
for q in mv.status[DebuggerStatus.NO_PROOF]:
    print('"' + mv[q].name_hash + '"', end=", ")

In [ ]:
dirs_to_send = []
additional = ""

for q in mv.fixable:
    r = mv.reviewers[q]
    if r.name_hash in LIKELY_FIXED:
        continue
    r.collect_garbage()
    dirs_to_send += r.get_dirs_to_sync()
    additional += f"\"{r.name_hash}\",\n"

print(additional)

for i, d in enumerate(dirs_to_send):
    if i % 5 == 0:
        print("echo '", (i //5 +1), "/", len(dirs_to_send)//5, "'")
    print("rsync -avz", d, "g2001:/home/yizhou7/mariposa/", "--inplace", "--delete", "--relative")

In [13]:
for q in mv.status[DebuggerStatus.FINISHED]:
    # print("./src/debugger3.py --collect-garbage -i", q)
    d = mv[q].sub_root
    print("rsync -avz", d, "g2001:/home/yizhou7/mariposa/", "--inplace", "--delete", "--relative")
    # print("./src/debugger3.py --build-trace-graph -i", q)
    # rev = mv.reviewers[q]
    # rev.collect_garbage()

rsync -avz dbg/f986501402 g2001:/home/yizhou7/mariposa/ --inplace --delete --relative
rsync -avz dbg/b546a78766 g2001:/home/yizhou7/mariposa/ --inplace --delete --relative
rsync -avz dbg/2a3b1202bd g2001:/home/yizhou7/mariposa/ --inplace --delete --relative
rsync -avz dbg/1b7f64e95b g2001:/home/yizhou7/mariposa/ --inplace --delete --relative
rsync -avz dbg/72d4708532 g2001:/home/yizhou7/mariposa/ --inplace --delete --relative
rsync -avz dbg/36a1859f27 g2001:/home/yizhou7/mariposa/ --inplace --delete --relative
rsync -avz dbg/243ec915e0 g2001:/home/yizhou7/mariposa/ --inplace --delete --relative
rsync -avz dbg/c1a05b62ba g2001:/home/yizhou7/mariposa/ --inplace --delete --relative
rsync -avz dbg/0b346d0f87 g2001:/home/yizhou7/mariposa/ --inplace --delete --relative
rsync -avz dbg/83448eecf5 g2001:/home/yizhou7/mariposa/ --inplace --delete --relative
rsync -avz dbg/1d57b073cc g2001:/home/yizhou7/mariposa/ --inplace --delete --relative
rsync -avz dbg/fae7756eb4 g2001:/home/yizhou7/mariposa